In [1]:
import tdmclient.notebook
await tdmclient.notebook.start()

In [2]:

import cv2
import math

import numpy as np
import matplotlib.pyplot as plt
from numpy.core.fromnumeric import size
from matplotlib import colors
from asgiref.sync import sync_to_async #in order to load variables to communicate between sync and async

import optimal_path as op

from robot import Robot
from Map import Map
from Local_navigation import* 
from Mouvement import*
from vision import*
from optimal_path import* 


map_lenght = 1
nb_of_square_by_side = 50
current = 1

global move 
move = False #bool which indicate how our robot is moving (1: avoid an object VS 0: following the optimal path)
kalman_bool = False
old_distance = 0
old_angle = 0
#pourcentage_reduc = 1

In [3]:

George = Robot()
Lausanne = Map(map_lenght, nb_of_square_by_side)

#VideoCap = vision_initialization()
VideoCap=cv2.VideoCapture(0)


In [4]:
@tdmclient.notebook.sync_var
def motors(l_speed=500, r_speed=500, verbose=False):
    """
    Sets the motor speeds of the Thymio 
    param l_speed: left motor speed
    param r_speed: right motor speed
    param verbose: whether to print status messages or not
    """
    global motor_left_target, motor_right_target
    # Printing the speeds if requested
    if verbose:
        print("\t\t Setting speed : ", l_speed, r_speed)
    motor_left_target = l_speed
    motor_right_target = r_speed

In [5]:
#############################
#Test MASK  A LANCER QUE POUR FAIRE DES TESTS SUR LE MASK
#############################

while True:
    ret, frame = VideoCap.read()
    mask, mask2 = mask_map_init(frame)
    
    keyVal = cv2.waitKey(1) & 0xFF
    if keyVal == ord('q'):
        break
    cv2.imshow("mask",mask)


In [ ]:
#np.set_printoptions(threshold=sys.maxsize)
####################################
## INIT
####################################

# Prend une première image
ret, frame = VideoCap.read()
print("taille frame", frame.shape)

print("MAP INITIALISATION")
# Set the pourcentage value between the nb of pixel and nb of square by side
Lausanne.set_map_lenght(frame)
#print("Pourcentage",Lausanne.get_pourcentage())
# Set the robot goal, position and angle
goal = init_goal(frame, Lausanne.get_pourcentage())
#goal = (50, 40)
pos_robot = George.get_pos()
#print("GOAL", goal)


#pos_robot, angle = update(frame, Lausanne.get_pourcentage())  # fonction qui retourne la position angle etc....
 
      #Utiliser au cas où il ne trouve pas la bonne position du premier coup
while (pos_robot[0] <= 0) & (pos_robot[1] <= 0):
    #ret, frame=VideoCap.read()
    #display(frame,1,0,0)
    ret, frame=VideoCap.read()
    #vs.display (frame, 0, 1, 1, 0, 5, Lausanne.get_pourcentage())
    pos_robot, angle = update(frame, Lausanne.get_pourcentage())  # fonction qui retourne la position angle etc....
    #print("Waiting robot position") 

George.set_angle(angle)
George.set_goal(goal)
George.set_start_pos(pos_robot)
George.set_pos(pos_robot)
print ("goal", goal)
print ("start position", pos_robot)

# SET MAP
Lausanne.set_map_lenght(frame)
#print("Lausanna lenght", Lausanne.get_lenght())

# MASK 
mask_bleu, mask_green = mask_map_init(frame)
#cv2.imshow("mask", mask)
cv2.imwrite("masque1.jpg", mask_bleu)
cv2.imwrite("masque2.jpg", mask_green)

# Fin de l'init
Lausanne.init_grid(mask_bleu,mask_green)
grille = Lausanne.get_map()
#cv2.imshow("grid", grille)
cv2.imwrite("grille.jpg", grille)

# OPTIMAL PATH
print("PATH COMPUTATION")
path, visitedNodes = op.path_computation(George.get_start() , George.get_goal() , Lausanne.get_lenght(), grille)

George.set_path(path) 
George.set_visit_nodes(visitedNodes) 

# Display the optimal path
op.display_map(Lausanne.get_lenght(),  grille,  George.get_visit_nodes(), George.get_path(), George.get_start(), George.get_goal())
#print("PATH", George.get_path())

display (frame, 0, 1, 1, 1, George.get_path(), Lausanne.get_pourcentage())
#print("PATH22222", George.get_path())
cv2.imwrite("TOTAL.jpg", frame)

#vision_end(VideoCap)



taille frame (480, 640, 3)
MAP INITIALISATION
No parking slot free


In [7]:

####################################
## while
####################################

George.set_current(0)
current= 0
kalman_bool = False
print("PATH", George.get_path())
old_distance = 0
old_angle = 0

print('START MOUVEMENT')
while True:#(George.get_pos() != George.get_goal()):
    
    # Read the camera
    ret, frame=VideoCap.read()
    
    # Recupere la position et angle --> XAV
    pos_robot,angle_robot = update(frame, Lausanne.get_pourcentage())
    George.set_pos(pos_robot)
    George.set_angle(angle_robot)
    
    if(current < George.get_path().shape[1]-1):
        current = path_update(pos_robot, 3, np.transpose(George.get_path())[current], George.get_current())
        George.set_current(current)
    else:
        print("ARRIVED")
        motors(0,0)
        break
    
    #print("pos", pos_robot)
    print("taille", George.get_path().shape[1])
    print("current", current)
    pos_goal =  np.transpose(George.get_path())[current]
    speed_l, speed_r, old_distance, old_goal  = move_to_position(pos_robot, angle_robot, pos_goal, old_distance, old_angle)

    motors(speed_l, speed_r) 
    
    display (frame, 0, 1, 1, 1, path, Lausanne.get_pourcentage())
    
    # Quit when we press q
    keyVal = cv2.waitKey(1) & 0xFF
    if keyVal == ord('q'):
        motors(0,0)
        break
     
    sleep(0.1)


print("The End")
    

PATH [[38 37 37 36 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35
  35 35 35 35 35 35 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18
  17 17 17 17 17 17 17]
 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
  25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 40 40 40 40 40 40 40 40
  39 38 37 36 35 34 33]]
START MOUVEMENT
PATH [38  1]
X 0
Y 0
next goal plz
taille 55
current 1
movement
(38, 1)
old dist, 0 old angle 0
debut 38 1 2.435166597933737
dist_debut 1.4142135623730951
angle_goal, 2.356194490192345
-0.07897210774139207
alpha -0.07897210774139207
v 2.8284271247461903
w -11.05609508379489
new dist, 1.4142135623730951 new angle -0.07897210774139207
v2 2.8284271247461903
speed_original, 103 81
speed, 103 81
PATH [37  2]
X 1
Y 1
next goal plz
taille 55
current 2
movement
(38, 1)
old dist, 1.4142135623730951 old angle 0
debut 38 1 2.4295918160939567
dist_debut 2.23606797749979
angle_goal, 2.0344439357957027
-0.395147880298254
alpha -0.3951478802

PATH [36  4]
X 1
Y 4
taille 55
current 3
movement
(35, 0)
old dist, 4.123105625617661 old angle 0
debut 35 0 1.631944489444198
dist_debut 4.123105625617661
angle_goal, 1.3258176636680326
-0.30612682577616535
alpha -0.30612682577616535
v 8.246211251235321
w -42.85775560866315
new dist, 4.123105625617661 new angle -0.30612682577616535
v2 8.246211251235321
speed_original, 42 -42
speed, 42 -42
PATH [36  4]
X 1
Y 4
taille 55
current 3
movement
(35, 0)
old dist, 4.123105625617661 old angle 0
debut 35 0 1.5914119624205003
dist_debut 4.123105625617661
angle_goal, 1.3258176636680326
-0.26559429875246776
alpha -0.26559429875246776
v 8.246211251235321
w -37.183201825345485
new dist, 4.123105625617661 new angle -0.26559429875246776
v2 8.246211251235321
speed_original, 37 -37
speed, 37 -37
PATH [36  4]
X 1
Y 3
taille 55
current 3
movement
(35, 1)
old dist, 4.123105625617661 old angle 0
debut 35 1 1.560380036863927
dist_debut 3.1622776601683795
angle_goal, 1.2490457723982544
-0.3113342644656725
alph

PATH [36  4]
X 4
Y 2
taille 55
current 3
movement
(32, 2)
old dist, 4.47213595499958 old angle 0
debut 32 2 0.6435011087932843
dist_debut 4.47213595499958
angle_goal, 0.46364760900080615
-0.1798534997924781
alpha -0.1798534997924781
v 8.94427190999916
w -25.17948997094694
new dist, 4.47213595499958 new angle -0.1798534997924781
v2 8.94427190999916
speed_original, 25 -25
speed, 25 -25
PATH [36  4]
X 4
Y 1
taille 55
current 3
movement
(32, 3)
old dist, 4.47213595499958 old angle 0
debut 32 3 0.5880026035475677
dist_debut 4.123105625617661
angle_goal, 0.24497866312686423
-0.3430239404207035
alpha -0.3430239404207035
v 8.246211251235321
w -48.02335165889849
new dist, 4.123105625617661 new angle -0.3430239404207035
v2 8.246211251235321
speed_original, 48 -48
speed, 48 -48
PATH [36  4]
X 4
Y 1
taille 55
current 3
movement
(32, 3)
old dist, 4.123105625617661 old angle 0
debut 32 3 0.5642681888579317
dist_debut 4.123105625617661
angle_goal, 0.24497866312686423
-0.3192895257310675
alpha -0.3192

PATH [36  4]
X 4
Y 2
taille 55
current 3
movement
(32, 6)
old dist, 4.47213595499958 old angle 0
debut 32 6 -0.21634066137329047
dist_debut 4.47213595499958
angle_goal, 5.81953769817878
6.03587835955207
alpha -0.24730694762751568
v 8.94427190999916
w -34.622972667852196
new dist, 4.47213595499958 new angle -0.24730694762751568
v2 8.94427190999916
speed_original, 34 -34
speed, 34 -34
PATH [36  4]
X 3
Y 2
taille 55
current 3
movement
(33, 6)
old dist, 4.47213595499958 old angle 0
debut 33 6 -0.25020054777764095
dist_debut 3.605551275463989
angle_goal, 5.695182703632018
5.945383251409659
alpha -0.3378020557699269
v 7.211102550927978
w -47.29228780778976
new dist, 3.605551275463989 new angle -0.3378020557699269
v2 7.211102550927978
speed_original, 47 -47
speed, 47 -47
PATH [36  4]
X 3
Y 2
taille 55
current 3
movement
(33, 6)
old dist, 3.605551275463989 old angle 0
debut 33 6 -0.2914567944778674
dist_debut 3.605551275463989
angle_goal, 5.695182703632018
5.986639498109886
alpha -0.2965458090

PATH [36  4]
X 2
Y 3
taille 55
current 3
movement
(34, 7)
old dist, 3.605551275463989 old angle 0
debut 34 7 -0.7406519319428327
dist_debut 3.605551275463989
angle_goal, 5.3003915839322575
6.04104351587509
alpha -0.24214179130449598
v 7.211102550927978
w -33.899850782629436
new dist, 3.605551275463989 new angle -0.24214179130449598
v2 7.211102550927978
speed_original, 33 -33
speed, 33 -33
PATH [36  4]
X 2
Y 3
taille 55
current 3
movement
(34, 7)
old dist, 3.605551275463989 old angle 0
debut 34 7 -0.7406519319428327
dist_debut 3.605551275463989
angle_goal, 5.3003915839322575
6.04104351587509
alpha -0.24214179130449598
v 7.211102550927978
w -33.899850782629436
new dist, 3.605551275463989 new angle -0.24214179130449598
v2 7.211102550927978
speed_original, 33 -33
speed, 33 -33
PATH [36  4]
X 2
Y 3
taille 55
current 3
movement
(34, 7)
old dist, 3.605551275463989 old angle 0
debut 34 7 -0.7406519319428327
dist_debut 3.605551275463989
angle_goal, 5.3003915839322575
6.04104351587509
alpha -0.2

...

PATH [35  5]
X 2
Y 3
taille 55
current 4
movement
(33, 2)
old dist, 3.605551275463989 old angle 0
debut 33 2 1.3815973046949284
dist_debut 3.605551275463989
angle_goal, 0.982793723247329
-0.3988035814475993
alpha -0.3988035814475993
v 7.211102550927978
w -55.83250140266391
new dist, 3.605551275463989 new angle -0.3988035814475993
v2 7.211102550927978
speed_original, 55 -55
speed, 55 -55
PATH [35  5]
X 1
Y 4
taille 55
current 4
movement
(34, 1)
old dist, 3.605551275463989 old angle 0
debut 34 1 1.4347989087575126
dist_debut 4.123105625617661
angle_goal, 1.3258176636680326
-0.10898124508948004
alpha -0.10898124508948004
v 8.246211251235321
w -15.257374312527205
new dist, 4.123105625617661 new angle -0.10898124508948004
v2 8.246211251235321
speed_original, 113 82
speed, 113 82
PATH [35  5]
X 0
Y 5
taille 55
current 4
movement
(35, 10)
old dist, 4.123105625617661 old angle 0
debut 35 10 -1.5821594740591873
dist_debut 5.0
angle_goal, 4.71238898038469
6.294548454443877
alpha 0.01136314726429

PATH [35  7]
X 0
Y 3
taille 55
current 6
movement
(35, 4)
old dist, 3.1622776601683795 old angle 0
debut 35 4 1.8700426423973733
dist_debut 3.0
angle_goal, 1.5707963267948966
-0.2992463156024767
alpha -0.2992463156024767
v 6.0
w -41.894484184346744
new dist, 3.0 new angle -0.2992463156024767
v2 6.0
speed_original, 41 -41
speed, 41 -41
PATH [35  7]
X 0
Y 2
next goal plz
taille 55
current 7
movement
(35, 5)
old dist, 3.0 old angle 0
debut 35 5 1.860299998756285
dist_debut 3.0
angle_goal, 1.5707963267948966
-0.2895036719613884
alpha -0.2895036719613884
v 6.0
w -40.53051407459437
new dist, 3.0 new angle -0.2895036719613884
v2 6.0
speed_original, 40 -40
speed, 40 -40
PATH [35  8]
X 0
Y 3
taille 55
current 7
movement
(35, 5)
old dist, 3.0 old angle 0
debut 35 5 1.8307364669834247
dist_debut 3.0
angle_goal, 1.5707963267948966
-0.2599401401885282
alpha -0.2599401401885282
v 6.0
w -36.39161962639395
new dist, 3.0 new angle -0.2599401401885282
v2 6.0
speed_original, 36 -36
speed, 36 -36
PATH [35

PATH [35  9]
X 3
Y 3
taille 55
current 8
movement
(32, 6)
old dist, 4.242640687119285 old angle 0
debut 32 6 1.0927956365202698
dist_debut 4.242640687119285
angle_goal, 0.7853981633974483
-0.3073974731228215
alpha -0.3073974731228215
v 8.48528137423857
w -43.03564623719501
new dist, 4.242640687119285 new angle -0.3073974731228215
v2 8.48528137423857
speed_original, 43 -43
speed, 43 -43
PATH [35  9]
X 3
Y 3
taille 55
current 8
movement
(32, 6)
old dist, 4.242640687119285 old angle 0
debut 32 6 1.0267536539728521
dist_debut 4.242640687119285
angle_goal, 0.7853981633974483
-0.24135549057540384
alpha -0.24135549057540384
v 8.48528137423857
w -33.78976868055654
new dist, 4.242640687119285 new angle -0.24135549057540384
v2 8.48528137423857
speed_original, 33 -33
speed, 33 -33
PATH [35  9]
X 3
Y 3
taille 55
current 8
movement
(32, 6)
old dist, 4.242640687119285 old angle 0
debut 32 6 0.982793723247329
dist_debut 4.242640687119285
angle_goal, 0.7853981633974483
-0.19739555984988078
alpha -0.19

PATH [35  9]
X 4
Y 0
taille 55
current 8
movement
(31, 9)
old dist, 4.0 old angle 0
debut 31 9 0.17219081452293897
dist_debut 4.0
angle_goal, 0.0
-0.17219081452293897
alpha -0.17219081452293897
v 8.0
w -24.106714033211453
new dist, 4.0 new angle -0.17219081452293897
v2 8.0
speed_original, 122 73
speed, 122 73
PATH [35  9]
X 4
Y 1
taille 55
current 8
movement
(31, 10)
old dist, 4.0 old angle 0
debut 31 10 0.13888501552021276
dist_debut 4.123105625617661
angle_goal, 6.038206644052722
5.899321628532509
alpha -0.383863678647077
v 8.246211251235321
w -53.74091501059078
new dist, 4.123105625617661 new angle -0.383863678647077
v2 8.246211251235321
speed_original, 53 -53
speed, 53 -53
PATH [35  9]
X 4
Y 1
taille 55
current 8
movement
(31, 10)
old dist, 4.123105625617661 old angle 0
debut 31 10 0.0964737751825861
dist_debut 4.123105625617661
angle_goal, 6.038206644052722
5.941732868870136
alpha -0.3414524383094503
v 8.246211251235321
w -47.80334136332304
new dist, 4.123105625617661 new angle -0

PATH [35  9]
X 2
Y 3
taille 55
current 8
movement
(33, 12)
old dist, 3.605551275463989 old angle 0
debut 33 12 -0.7109093731835666
dist_debut 3.605551275463989
angle_goal, 5.3003915839322575
6.011300957115824
alpha -0.2718843500637621
v 7.211102550927978
w -38.06380900892669
new dist, 3.605551275463989 new angle -0.2718843500637621
v2 7.211102550927978
speed_original, 38 -38
speed, 38 -38
PATH [35  9]
X 2
Y 3
taille 55
current 8
movement
(33, 12)
old dist, 3.605551275463989 old angle 0
debut 33 12 -0.7109093731835666
dist_debut 3.605551275463989
angle_goal, 5.3003915839322575
6.011300957115824
alpha -0.2718843500637621
v 7.211102550927978
w -38.06380900892669
new dist, 3.605551275463989 new angle -0.2718843500637621
v2 7.211102550927978
speed_original, 38 -38
speed, 38 -38
PATH [35  9]
X 2
Y 4
taille 55
current 8
movement
(33, 13)
old dist, 3.605551275463989 old angle 0
debut 33 13 -0.7929168187097523
dist_debut 4.47213595499958
angle_goal, 5.176036589385496
5.968953408095248
alpha -0.

In [7]:
vision_end(VideoCap)

In [8]:
motors(0,0)